In [3]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY not found in .env")

openai = OpenAI(api_key=api_key)
MODEL = "gpt-4o-mini"

base_system_message = """You are a helpful assistant in a clothing store. 
You should gently encourage the customer to try items that are on sale. 
Hats are 60% off, and most other items are 50% off. 
If they mention hats, encourage them to explore those. 
If they ask about shoes, mention they're not on sale, but remind them about hats. 
If they ask for belts, inform them that belts aren't sold, and suggest sale items instead."""

def chat(message, history):
    system_message = base_system_message
    if "belt" in message.lower():
        system_message += " Remember: belts aren't available."

    messages = [{"role": "system", "content": system_message}]
    
    for turn in history:
        if turn["role"] == "user":
            messages.append({"role": "user", "content": turn["content"]})
        elif turn["role"] == "assistant":
            messages.append({"role": "assistant", "content": turn["content"]})

    messages.append({"role": "user", "content": message})

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        yield response

gr.ChatInterface(fn=chat, type="messages", title="🧢 Clothing Store Chatbot").launch()


* Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.
